<a href="https://colab.research.google.com/github/jason96819/Project_1/blob/main/%EB%84%A5%EC%8A%A8_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_230711_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

프로젝트 내용 : 이번 23.06.10일 넥슨 대표 게임인 메이플 스토리의 6차 전직이 발표되었다.

이에 대해 유저들의 반응을 알아보고자 데이터를 수집하고 반응들을 살펴보는 프로젝트를 진행하였다.

또한, 수집한 데이터를 바탕으로 어떤 지표를 볼 수 있는지 확인해 보겠습니다.

필요한 라이브러리 import

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.error

# 0. 데이터 수집

## 0.1. 메이플 인벤

https://maple.inven.co.kr/

어떠한 이유인지 모르겠지만, 게시물과 게시물의 내용은 크롤링은 수월하지만 댓글 크롤링은 빈칸으로 출력.

게시물의 시간, 게시물 제목, 내용만 크롤링 실시

6월 1일의 첫 게시물 페이지 링크 : https://www.inven.co.kr/board/maple/5974?name=subject&keyword=6%EC%B0%A8&sterm=8430942&p=5

6월 30일의 마지막 게시물 페이지 링크 : https://www.inven.co.kr/board/maple/5974?name=subject&keyword=6%EC%B0%A8&sterm=8180942&p=2

게시물 특성상 최근 게시물이 앞의 페이지에 있음. 데이터 수집한 뒤 날짜 오래된순으로 정렬

sterm의 방식은 818의 페이지 2부터 843의 페이지 5까지, 페이지의 수는 1부터 최대 ??페이지

### 모든 페이지의 게시물 링크 가져오기

5분 소요

In [ ]:
link_list = []

for i in range(818, 844):
    for j in range(1, 51):

        url = f"https://www.inven.co.kr/board/maple/5974?name=subject&keyword=6%EC%B0%A8&sterm={i}0942&p={j}"
        bsObject = BeautifulSoup(urlopen(url), "html.parser")

        no_result_div = bsObject.find('div', class_="no-result")

        if no_result_div:
            break
        else:
            for link in bsObject.find_all('a', attrs={"class":"subject-link"}):
                text = link.text.strip()
                href = link.get('href')
                if "[알림]" not in text:
                    link_list.append(href)

len(link_list)

10570

### 가져온 링크를 바탕으로 게시 시간, 게시물 제목, 게시물 내용 크롤링

In [ ]:
df = pd.DataFrame(columns=['Date', 'Reaction'])

In [ ]:
def insert_data(df, date_, reac):
    data = [date_, reac]
    df.loc[len(df)] = data

In [ ]:
def mk_df():

    for url in link_list:

        html = urlopen(url)
        bsObject = BeautifulSoup(html, "lxml")

        for dates in bsObject.find_all('div', attrs={"class":"articleDate"}):
            date = dates.text.strip()

        for titles in bsObject.find_all('div', attrs={"class":"articleTitle"}):
            title = titles.text.strip()

        content_divs = bsObject.find_all('div', attrs={"id": "powerbbsContent"})
        if len(content_divs) > 0:
            content = content_divs[0].text.strip()

        reaction = title + ' ' + content

        insert_data(df, date, reaction)

In [ ]:
mk_df()
df

HTTPError: ignored

53분 소요 후, 404에러 즉, 많은 양의 게시물을 스크랩하기에는 시간이 너무오래걸려 시간이 지날수록 페이지가 사라질 가능성이 있음.

## 0.2. 메이플 펨코

https://www.fmkorea.com/maple

In [3]:
text_list = []

url = "https://www.fmkorea.com/maple"

bsObject = BeautifulSoup(urlopen(url), "html.parser")

for txt in bsObject.find_all('td', attrs={"class":"cate"}):
    text = txt.text.strip()
    text_list.append(text)
text_list

HTTPError: ignored

펨코는 429에러로 인해 스크랩하기 힘듦.

따라서 다른 방법으로 메이플 인벤만 스크랩하기로 결정.